In [1]:
import numpy as np # We need numpy to handle matrices of numbers and data
import tensorflow as tf # Tensorflow is the library that we will be using for simulating an LSTM model
import pandas as pd # Pandas library are for reading .csv files or other excel files
from matplotlib import pyplot as plt # Matplotlib for visualizing data or plotting it
from sklearn.preprocessing import MinMaxScaler # This is for scaling the data 

In [2]:
# This code reads the csv file and saves it in a variable as a pandas data frame
df = pd.read_csv('../datas/train_passengers_copy.csv')

#Variables for training
cols = list(df)[1:3]
df_for_training = df[cols].astype(float)

#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = MinMaxScaler()
scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 5. We will make timesteps = 14 (past days data used for training). 

#Empty lists to be populated using formatted training data
trainX = []
trainY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 3  # Number of past days we want to use to predict the future.

In [3]:
#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (141, 3, 2).
trainY shape == (141, 1).


In [4]:
load_model = tf.keras.models.load_model("../saved_param/model2")

In [5]:
load_model.layers[1].trainable_weights

[<tf.Variable 'dense/kernel:0' shape=(3, 1) dtype=float32, numpy=
 array([[-0.8367793],
        [-0.4085069],
        [ 1.1929541]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.01844767], dtype=float32)>]

In [6]:
load_model.predict(np.array([trainX[0]]))

1/1 [==============================] - 3s 3s/step


array([[0.06236403]], dtype=float32)

In [7]:
trainX[0]

array([[0.01544402, 0.8       ],
       [0.02702703, 0.8       ],
       [0.05405405, 0.6       ]])

In [8]:
units = 3
W = load_model.layers[0].get_weights()[0]
U = load_model.layers[0].get_weights()[1]
b = load_model.layers[0].get_weights()[2]

W_i = W[:, :units]
W_f = W[:, units: units * 2]
W_c = W[:, units * 2: units * 3]
W_o = W[:, units * 3:]

U_i = U[:, :units]
U_f = U[:, units: units * 2]
U_c = U[:, units * 2: units * 3]
U_o = U[:, units * 3:]

b_i = b[:units]
b_f = b[units: units * 2]
b_c = b[units * 2: units * 3]
b_o = b[units * 3:]

In [9]:
print(W_i)
print(U_i)
print(b_i)


[[ 0.10793971  0.81825334  0.6556316 ]
 [-0.60634685 -0.6131416  -0.49448305]]
[[ 0.5622357   0.93071234  0.46856326]
 [ 0.37466365 -0.48716673 -0.1355884 ]
 [-0.52040434  0.1333436   0.87895346]]
[-0.22539224  0.01720281  0.21763435]


In [10]:
print(W_f)
print(U_f)
print(b_f)


[[-0.56221163 -0.01619006 -0.14644638]
 [-0.3923156   0.25872135  0.4800443 ]]
[[-0.02404333 -0.03320475  0.2481951 ]
 [-0.16247411 -0.07040676  0.17257127]
 [-0.07867586  0.20210117 -0.24236679]]
[0.7972173 1.0329572 1.1679661]


In [11]:
print(W_c)
print(U_c)
print(b_c)

[[ 0.34074333 -0.36218873  0.87516445]
 [-0.16738938 -0.27611265 -0.06332847]]
[[ 0.12381045 -0.23511174  0.4601762 ]
 [ 0.43099013  0.2494406  -0.18612424]
 [ 0.00344216 -0.25496334  0.04300574]]
[-0.02486727 -0.06337232 -0.04763215]


In [12]:
print(W_o)
print(U_o)
print(b_o)


[[-0.3573572   0.20927963  0.44177994]
 [-0.27215135  0.3107963   0.27268055]]
[[ 0.12031795  0.30339935  0.0822956 ]
 [ 0.24917188  0.4231973  -0.43854418]
 [ 0.12620583  0.04300426 -0.1991784 ]]
[-0.21706288  0.09333088  0.22692391]
